### Packages

In [1]:
cd '/h/ama/workspace/ama-at-vector/best-mix'

/scratch/ssd001/home/ama/workspace/ama-at-vector/best-mix


In [17]:
from collections import OrderedDict
args = {'dataset': 'cifar10', 'data_dir': '/h/ama/workspace/ama-at-vector/best-mix/data', 'root_dir': '/scratch/hdd001/home/ama/mixup/2022-07-02/11', 'labels_per_class': 5000, 'valid_labels_per_class': 0, 'arch': 'preactresnet18', 'initial_channels': 64, 'epochs': 300, 'method': 'vanilla', 'train': 'vanilla', 'in_batch': False, 'mixup_alpha': 1.0, 'dropout': False, 'box': False, 'graph': False, 'neigh_size': 4, 'n_labels': 3, 'beta': 1.2, 'gamma': 0.5, 'eta': 0.2, 'transport': True, 't_eps': 0.8, 't_size': -1, 'adv_eps': 10.0, 'adv_p': 0.0, 'clean_lam': 0.0, 'mp': 8, 'batch_size': 100, 'learning_rate': 0.2, 'momentum': 0.9, 'decay': 0.0001, 'schedule': [100, 200], 'gammas': [0.1, 0.1], 'print_freq': 100, 'resume': '', 'start_epoch': 0, 'evaluate': False, 'ngpu': 1, 'workers': 2, 'seed': 1, 'add_name': '', 'log_off': False, 'job_id': 7889109, 'enable_wandb': False, 'wandb_project': 'test', 'job_name': '11', 'blur_sigma': 1.0, 'kernel_size': 5, 'grad_normalization': 'L1', 'eval_mode': False, 'new_implementation': True, 'with_shift': True, 'use_yp_argmax': False, 'mix_stride': 2, 'rand_pos': 1, 'prob_mix': 1.0, 'mix_schedule': 'fixed', 'mix_scheduled_epoch': 50, 'upper_lambda': 0.5, 'mixup_alpha2': 0.0, 'use_cuda': True}

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

# mydict = {'val':'it works'}
# nested_dict = {'val':'nested works too'}
args = dotdict(args)

In [18]:
args.dataset='cifar10'
args.num_classes=10

In [19]:
log = None

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import pickle

from mixup import mixup_graph
import time
from utils_mixup import gradmix_v2, gradmix_v2_improved

os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline

In [22]:
''' Model '''
import models
# import torchvision.models as models
from load_data import load_data_subset
from collections import OrderedDict



resnet = models.__dict__['preactresnet18'](10, False, 1).cuda()

# checkpoint = torch.load('/group-volume/Multimodal-Learning/ssl/vse_files/runs/fast_autoaugment/models/cifar10_preact_ckpt/vanilla.pth.tar')
checkpoint = torch.load('checkpoint/cifar10_preact_ckpt_vanilla.pth.tar')

od = OrderedDict()
for key in checkpoint['state_dict'].keys():
    od[key[7:]] = checkpoint['state_dict'][key]
resnet.load_state_dict(od)

# resnet = models.resnet18(pretrained=True)

optimizer = torch.optim.SGD(resnet.parameters(),
                                0.2,
                                momentum=0.9,
                                weight_decay=1e-3,
                                nesterov=True)



# resnet.load_state_dict(checkpoint['state_dict'])
# mean = torch.tensor([x / 255 for x in [125.3, 123.0, 113.9]],dtype=torch.float32).reshape(1, 3, 1, 1).cuda()
# std = torch.tensor([x / 255 for x in [63.0, 62.1, 66.7]], dtype=torch.float32).reshape(1, 3, 1, 1).cuda()
labels_per_class = 5000
# mean = torch.tensor([125.3, 123.0, 113.9])/255
# std = torch.tensor([63.0, 62.1, 66.7])/255
# mean_torch = mean.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
# std_torch = std.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)

criterion = nn.CrossEntropyLoss()


batch_size = 100
workers=2
dataset='cifar10'
# data_dir='/group-volume/Multimodal-Learning/ssl/vse_files/runs/fast_autoaugment/data'
data_dir='data'
valid_labels_per_class=0
mixup_alpha=0
train_loader, valid_loader, _, test_loader, num_classes = load_data_subset(batch_size,workers,dataset,data_dir,labels_per_class=labels_per_class,valid_labels_per_class=valid_labels_per_class,mixup_alpha=mixup_alpha)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
from __future__ import division
# Codes are borrowed from https://github.com/vikasverma1077/manifold_mixup/tree/master/supervised

import os, sys, shutil, time, random
from collections import OrderedDict

sys.path.append('..')
if sys.version_info[0] < 3:
    import cPickle as pickle
else:
    import _pickle as pickle
import argparse
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable
from load_data import load_data_subset
from logger import plotting, copy_script_to_folder, AverageMeter, RecorderMeter, time_string, convert_secs2time
import models
from multiprocessing import Pool

import ipdb
from utils_log import wandbLogger, saveCheckpoint
import torchvision
import torchvision.transforms as transforms
from utils_mixup import gradmix, reweighted_lam, gradmix_v2, gradmix_v2_improved
from mixup import to_one_hot, get_lambda


def train(train_loader, model, optimizer, epoch, args, log, mp=None):
    '''train given model and dataloader'''
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    mixing_avg = []
    prob_mix = get_prob_mix(args.mix_schedule, args.prob_mix, epoch, args.mix_scheduled_epoch)

    # if args.blur_sigma != 0.:
    blurrer = transforms.GaussianBlur(kernel_size=(args.kernel_size, args.kernel_size),
                                      sigma=(args.blur_sigma, args.blur_sigma))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        input = input.cuda()
        target = target.long().cuda()

        unary = None
        noise = None
        adv_mask1 = 0
        adv_mask2 = 0

        # train with clean images
        if args.train == 'vanilla':
            input_var, target_var = Variable(input), Variable(target)
            output, reweighted_target = model(input_var, target_var)
            loss = bce_loss(softmax(output), reweighted_target)
#             break

        # train with mixup images
        elif args.train == 'mixup':

            batch_size = input.shape[0]
            mix_size = int(batch_size*prob_mix)

            # if mix_size is 0, we are simply doing standard training with no DA
            if mix_size == 0:
                input_var, target_var = Variable(input), Variable(target)
                output, reweighted_target = model(input_var, target_var)

                loss = bce_loss(softmax(output), reweighted_target)
            else:
                if mix_size == batch_size:
                    # entire batch is DA
                    input_2b_mixed = input
                    target_2b_mixed = target
                    input_std = None
                    target_std = None
                else:
                    # some inputs are augmented, some are not
                    input_std, input_2b_mixed = input[:(batch_size-mix_size)], input[(batch_size-mix_size):]
                    target_std, target_2b_mixed = target[:(batch_size-mix_size)], target[(batch_size-mix_size):]

                # process for Puzzle Mix
                if args.graph:
                    # whether to add adversarial noise or not
                    if args.adv_p > 0:
                        adv_mask1 = np.random.binomial(n=1, p=args.adv_p)
                        adv_mask2 = np.random.binomial(n=1, p=args.adv_p)
                    else:
                        adv_mask1 = 0
                        adv_mask2 = 0

                    # random start:
                    if (adv_mask1 == 1 or adv_mask2 == 1):
                        noise = torch.zeros_like(input_2b_mixed).uniform_(-args.adv_eps / 255.,
                                                                 args.adv_eps / 255.)
                        input_2b_mixed_orig = input_2b_mixed * args.std + args.mean
                        input_2b_mixed_noise = input_2b_mixed_orig + noise
                        input_2b_mixed_noise = torch.clamp(input_2b_mixed_noise, 0, 1)
                        noise = input_2b_mixed_noise - input_2b_mixed_orig
                        input_2b_mixed_noise = (input_2b_mixed_noise - args.mean) / args.std
                        input_2b_mixed_var = Variable(input_2b_mixed_noise, requires_grad=True)
                    else:
                        input_2b_mixed_var = Variable(input_2b_mixed, requires_grad=True)
                    target_2b_mixed_var = Variable(target_2b_mixed)

                    # calculate saliency (unary)
                    if args.clean_lam == 0:
                        model.eval()
                        output_mix = model(input_2b_mixed_var)
                        loss_batch = criterion_batch(output_mix, target_2b_mixed_var)
                    else:
                        model.train()
                        output_mix = model(input_2b_mixed_var)
                        loss_batch = 2 * args.clean_lam * criterion_batch(output_mix,
                                                                          target_2b_mixed_var) / args.num_classes

                    loss_batch_mean = torch.mean(loss_batch, dim=0)
                    loss_batch_mean.backward(retain_graph=True)

                    unary = torch.sqrt(torch.mean(input_2b_mixed_var.grad**2, dim=1))

                    # calculate adversarial noise
                    if (adv_mask1 == 1 or adv_mask2 == 1):
                        noise += (args.adv_eps + 2) / 255. * input_2b_mixed_var.grad.sign()
                        noise = torch.clamp(noise, -args.adv_eps / 255., args.adv_eps / 255.)
                        adv_mix_coef = np.random.uniform(0, 1)
                        noise = adv_mix_coef * noise

                    if args.clean_lam == 0:
                        model.train()
                        optimizer.zero_grad()

                input_2b_mixed_var, target_2b_mixed_var = Variable(input_2b_mixed), Variable(target_2b_mixed)
                output_mix, reweighted_target = model(input_2b_mixed_var,
                                                      target_2b_mixed_var,
                                                      mixup=True,
                                                      args=args,
                                                      grad=unary,
                                                      noise=noise,
                                                      adv_mask1=adv_mask1,
                                                      adv_mask2=adv_mask2,
                                                      mp=mp)
                if input_std is None:
                    # perform mixup and calculate loss
                    loss = bce_loss(softmax(output_mix), reweighted_target)
                else:
                    loss_mix = bce_loss_sum(softmax(output_mix), reweighted_target)

                    input_std_var, target_std_var = Variable(input_std), Variable(target_std)
                    output_std, reweighted_target_std = model(input_std_var, target_std_var)

                    loss_std = bce_loss_sum(softmax(output_std), reweighted_target_std)
                    loss = (loss_std+loss_mix)/batch_size/args.num_classes
#             break

        # integrating our method :AVery
        elif args.train == 'ours':

            batch_size = input.shape[0]
            mix_size = int(batch_size*prob_mix)

            # if mix_size is 0, we are simply doing standard training with no DA
            if mix_size == 0:
                input_var, target_var = Variable(input), Variable(target)
                output, reweighted_target = model(input_var, target_var)

                loss = bce_loss(softmax(output), reweighted_target)
            else:
                if mix_size == batch_size:
                    # entire batch is DA
                    input_2b_mixed = input
                    target_2b_mixed = target
                    input_std = None
                    target_std = None
                else:
                    # some inputs are augmented, some are not
                    input_std, input_2b_mixed = input[:(batch_size-mix_size)], input[(batch_size-mix_size):]
                    target_std, target_2b_mixed = target[:(batch_size-mix_size)], target[(batch_size-mix_size):]

                input_2b_mixed_var = Variable(input_2b_mixed, requires_grad=True)
                target_2b_mixed_var = Variable(target_2b_mixed)

                # calculate saliency
                if args.eval_mode:
                    model.eval()
                else:
                    model.train()
                output = model(input_2b_mixed_var)

                if args.use_yp_argmax:
                    loss_batch = criterion_batch(output, output.argmax(dim=1))
                else:
                    loss_batch = criterion_batch(output, target_2b_mixed_var)

                loss_batch_mean = torch.mean(loss_batch, dim=0)
                loss_batch_mean.backward(retain_graph=True)

                model.train()

                g = input_2b_mixed_var.grad.data.abs().mean(dim=1, keepdim=True).detach()

                # apply gaussian bluring to the gradients
#                 if args.blur_sigma == 100:
#                     _blur_sigma = np.random.uniform(low=1.0, high=2.0)
#                 else:
#                     _blur_sigma = args.blur_sigma
#                 blurrer = transforms.GaussianBlur(kernel_size=(args.kernel_size, args.kernel_size),
#                                                   sigma=(_blur_sigma, _blur_sigma))
                g_tilde = blurrer(g)

                if args.with_shift:
                    if args.mixup_alpha2 == 0.:
                        if args.mixup_alpha == 0.:
                            sampled_alpha = 0.5
                        else:
                            sampled_alpha = get_lambda(args.mixup_alpha)
                        sampled_alpha *= args.upper_lambda
                    else:
                        sampled_alpha = get_lambda(args.mixup_alpha, args.mixup_alpha2)
                    
#                     if args.arch == 'resnext29_4_24':
                    mixed_x, mixed_y, mixed_lam = gradmix_v2_improved(input_2b_mixed_var,
                                                                     target_2b_mixed_var,
                                                                     g_tilde,
                                                                     alpha=sampled_alpha,
                                                                     normalization=args.grad_normalization,
                                                                     stride=args.mix_stride,
                                                                     debug=False,
                                                                     rand_pos=args.rand_pos)
                else:
                    mixed_x, mixed_y, mixed_lam = gradmix(input_2b_mixed_var,
                                                          target_2b_mixed_var,
                                                          g_tilde)
                optimizer.zero_grad()

                if args.new_implementation:
                    reweighted_target_mix = reweighted_lam(mixed_y, mixed_lam, args.num_classes)

                    if input_std is None:
                        output_mix = model(mixed_x)
                        loss = bce_loss(softmax(output_mix), reweighted_target_mix)
                    else:
                        input_std_var, target_std_var = Variable(input_std), Variable(target_std)
                        input_concat = torch.cat([mixed_x, input_std_var], dim=0)

                        output_mix = model(input_concat)

                        reweighted_target_std = to_one_hot(target_std_var, args.num_classes)
                        reweighted_target_concat = torch.cat([reweighted_target_mix, reweighted_target_std], dim=0)

                        target_concat = torch.cat([target_2b_mixed, target_std], dim=0)

                        loss = bce_loss(softmax(output_mix), reweighted_target_concat)
                else:
                    # perform mixup and calculate loss
                    reweighted_target = reweighted_lam(mixed_y, mixed_lam, args.num_classes)
                    output_mix = model(mixed_x)
                    if input_std is None:
                        loss = bce_loss(softmax(output_mix), reweighted_target)
                    else:
                        loss_mix = bce_loss_sum(softmax(output_mix), reweighted_target)

                        input_std_var, target_std_var = Variable(input_std), Variable(target_std)
                        output_std, reweighted_target_std = model(input_std_var, target_std_var)

                        loss_std = bce_loss_sum(softmax(output_std), reweighted_target_std)

                        loss = (loss_std+loss_mix)/batch_size/args.num_classes

    ########

        # for manifold mixup
        elif args.train == 'mixup_hidden':
            batch_size = input.shape[0]
            mix_size = int(batch_size*prob_mix)

            # if mix_size is 0, we are simply doing standard training with no DA
            if mix_size == 0:
                input_var, target_var = Variable(input), Variable(target)
                if args.arch == 'resnext29_4_24':
                    output = model(input_var)
                    reweighted_target = to_one_hot(target_var, args.num_classes)
                else:
                    output, reweighted_target = model(input_var, target_var)

                loss = bce_loss(softmax(output), reweighted_target)
            else:
                if mix_size == batch_size:
                    # entire batch is DA
                    input_2b_mixed = input
                    target_2b_mixed = target
                    input_std = None
                    target_std = None
                else:
                    # some inputs are augmented, some are not
                    input_std, input_2b_mixed = input[:(batch_size-mix_size)], input[(batch_size-mix_size):]
                    target_std, target_2b_mixed = target[:(batch_size-mix_size)], target[(batch_size-mix_size):]

                input_2b_mixed_var, target_2b_mixed_var = Variable(input_2b_mixed), Variable(target_2b_mixed)
                output_mix, reweighted_target = model(input_2b_mixed_var, target_2b_mixed_var, mixup_hidden=True, args=args)

                if input_std is None:
                    # perform mixup and calculate loss
                    loss = bce_loss(softmax(output_mix), reweighted_target)
                else:
                    loss_mix = bce_loss_sum(softmax(output_mix), reweighted_target)

                    input_std_var, target_std_var = Variable(input_std), Variable(target_std)
                    if args.arch == 'resnext29_4_24':
                        output_std = model(input_std_var)
                        reweighted_target_std = to_one_hot(target_std_var, args.num_classes)
                    else:
                        output_std, reweighted_target_std = model(input_std_var, target_std_var)

                    loss_std = bce_loss_sum(softmax(output_std), reweighted_target_std)
                    loss = (loss_std+loss_mix)/batch_size/args.num_classes
#             break
        else:
            raise AssertionError('wrong train type!!')

        # measure accuracy and record loss
        if args.train in ['mixup', 'mixup_hidden']:
            if mix_size == 0:
                prec1, prec5 = accuracy(output, target, topk=(1, 5))
            elif mix_size == batch_size:
                prec1, prec5 = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
            else:
                prec1_mix, prec5_mix = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
                prec1_std, prec5_std = accuracy(output_std, target_std, topk=(1, 5))
                prec1 = prec1_mix + prec1_std
                prec5 = prec5_mix + prec5_std
        elif args.train == 'ours':
            if mix_size == 0:
                prec1, prec5 = accuracy(output, target, topk=(1, 5))
            elif mix_size == batch_size:
                prec1, prec5 = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
            else:
                if args.new_implementation:
                    prec1, prec5 = accuracy(output_mix, target_concat, topk=(1, 5))
                else:
                    prec1_mix, prec5_mix = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
                    prec1_std, prec5_std = accuracy(output_std, target_std, topk=(1, 5))
                    prec1 = prec1_mix + prec1_std
                    prec5 = prec5_mix + prec5_std
        elif args.train == 'vanilla':
            prec1, prec5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # compute gradient and do SGD step
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    print_log(
        '  **Train** Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f} Error@1 {error1:.3f}'.format(
            top1=top1, top5=top5, error1=100 - top1.avg), log)
    return top1.avg, top5.avg, losses.avg

def get_prob_mix(mix_schedule, max_prob, epoch, scheduled_epoch):
    """
    with scheldued mix,
    if epoch < scheduled_epoch, we linearly increase the mix prob from 0 to max_prob
    if epoch > scheduled_epoch, we fix the prob at max_prob
    """
    if mix_schedule == 'fixed':
        prob_mix = max_prob
    elif mix_schedule == 'scheduled':
        if epoch+1>=scheduled_epoch:
            prob_mix = max_prob
        else:
            prob_mix = (epoch+1)/scheduled_epoch*max_prob
    elif mix_schedule == 'delayed':
        if epoch>=scheduled_epoch:
            prob_mix = max_prob
        else:
            prob_mix = 0
    return prob_mix

bce_loss = nn.BCELoss().cuda()
bce_loss_sum = nn.BCELoss(reduction='sum').cuda()
softmax = nn.Softmax(dim=1).cuda()
criterion = nn.CrossEntropyLoss().cuda()
criterion_batch = nn.CrossEntropyLoss(reduction='none').cuda()

def accuracy(output, target, topk=(1, )):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def print_log(print_string, log, end='\n'):
    '''print log'''
    print("{}".format(print_string), end=end)
    if log is not None:
        if end == '\n':
            log.write('{}\n'.format(print_string))
        else:
            log.write('{} '.format(print_string))
        log.flush()

In [9]:
def change_args(args):
    if args.method == 'vanilla':
        args.train = 'vanilla'
    elif args.method == 'input':
        args.train = 'mixup'
        if args.dataset in ['cifar10', 'cifar100']:
            args.mixup_alpha = 1.0
        else:
            args.mixup_alpha = 0.2
    elif args.method == 'manifold':
        args.train = 'mixup_hidden'
        if args.dataset in ['cifar10', 'cifar100']:
            args.mixup_alpha = 2.0
        else:
            args.mixup_alpha = 0.2
    elif args.method == 'cutmix':
        args.train = 'mixup'
        args.box = True
        if args.dataset in ['cifar10', 'cifar100']:
            args.mixup_alpha = 1.0
        else:
            args.mixup_alpha = 0.2
    elif args.method == 'puzzle':
        args.train = 'mixup'
        args.graph = True
        args.mixup_alpha = 1.0
        args.n_labels = 3
        args.eta = 0.2
        args.beta = 1.2
        args.gamma = 0.5
        args.neigh_size = 4
        args.transport = True
        if args.dataset in ['cifar10', 'cifar100']:
            args.t_size = 4
        args.t_eps = 0.8
        if args.dataset == 'tiny-imagenet-200':
            args.clean_lam = 1
    elif args.method == 'ours':
        args.train = 'ours'
    return args

### Model, Data, Saliency

In [17]:
args.method = 'vanilla'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


vanilla
  **Train** Prec@1 73.414 Prec@5 97.780 Error@1 26.586
Average total time: 30.8060 seconds (var: 0.0000)
Max/Min time: 30.8060/30.8060 seconds


In [18]:
args.method = 'input'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


mixup
  **Train** Prec@1 44.088 Prec@5 80.734 Error@1 55.912
Average total time: 31.0123 seconds (var: 0.0000)
Max/Min time: 31.0123/31.0123 seconds


In [20]:
args.method = 'cutmix'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


mixup
  **Train** Prec@1 48.766 Prec@5 88.740 Error@1 51.234
Average total time: 31.1112 seconds (var: 0.0000)
Max/Min time: 31.1112/31.1112 seconds


In [21]:
args.method = 'manifold'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


mixup_hidden
  **Train** Prec@1 43.182 Prec@5 84.238 Error@1 56.818
Average total time: 31.4363 seconds (var: 0.0000)
Max/Min time: 31.4363/31.4363 seconds


In [22]:
args.method = 'puzzle'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


mixup
  **Train** Prec@1 49.922 Prec@5 89.536 Error@1 50.078
Average total time: 59.6250 seconds (var: 0.0000)
Max/Min time: 59.6250/59.6250 seconds


In [26]:
args.method = 'ours'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


ours
  **Train** Prec@1 22.682 Prec@5 69.154 Error@1 77.318
Average total time: 103.7237 seconds (var: 0.0000)
Max/Min time: 103.7237/103.7237 seconds


In [16]:
args.method = 'ours'
args = change_args(args)
print(args.train)

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


ours
  **Train** Prec@1 22.566 Prec@5 69.608 Error@1 77.434
Average total time: 104.4569 seconds (var: 0.0000)
Max/Min time: 104.4569/104.4569 seconds
